In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils import resample

# 1. تحميل مجموعة البيانات
file_path = r'C:\Users\Nasee\Downloads\spambase.csv'
data = pd.read_csv(file_path)

# 2. التحقق من القيم المفقودة
missing_values = data.isnull().sum()
print("Missing values:")
print(missing_values)

# 3. اختيار العمود الهدف
label_column = data.columns[-1]  # اختر آخر عمود كهدف

# تنظيف العمود الهدف
data[label_column] = pd.to_numeric(data[label_column], errors='coerce')  # تحويل القيم إلى أرقام
data = data.dropna(subset=[label_column])  # حذف الصفوف التي تحتوي على NaN في العمود الهدف

# التحقق من عدد الصفوف المتبقية بعد التنظيف
print(f"Number of rows after cleaning: {data.shape[0]}")

# 4. توازن الفئات (إذا كانت هناك فئات غير متوازنة)
# تقسيم البيانات إلى فئات
df_majority = data[data[label_column] == 0]
df_minority = data[data[label_column] == 1]

# توازن البيانات باستخدام تقنية الزيادة (oversampling)
df_minority_upsampled = resample(df_minority, 
                                   replace=True,     # إعادة التكرار
                                   n_samples=len(df_majority),  # عدد العينات في الفئة الكبرى
                                   random_state=42)  # للتحكم في النتيجة

# دمج الفئات
data_balanced = pd.concat([df_majority, df_minority_upsampled])

# 5. تقسيم البيانات إلى مجموعة التدريب والاختبار
X = data_balanced.drop(columns=[label_column])  # استبعاد العمود الهدف
y = data_balanced[label_column]  # العمود الهدف

# تحويل البيانات النصية إلى نصوص
data_balanced['text'] = data_balanced.apply(lambda row: ' '.join([f'{col}:{row[col]}' for col in X.columns]), axis=1)

# تقسيم البيانات
X_train, X_test, y_train, y_test = train_test_split(data_balanced['text'], y, test_size=0.2, random_state=42)

# 6. إعداد TfidfVectorizer مع تحسينات
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 7. تدريب نموذج نايف بايز
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# 8. تقييم النموذج
y_pred = model.predict(X_test_tfidf)

# 9. حساب الدقة
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# 10. طباعة تقرير التصنيف
print("Classification Report:")
print(classification_report(y_test, y_pred))


Missing values:
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
56    0
57    0
dtype: int64
Number of rows after cleaning: 4601
Accuracy: 0.83
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.91      0.84       578
           1       0.88      0.74      0.80       538

    accuracy                           0.83      1116
   macro avg       0.84      0.82      0.82      1116
weighted avg       0.83      0.83      0.83      1116

